In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer

In [3]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [5]:
sen_df = spark.createDataFrame([
    (0,'Hi I heard about spark'),
    (1,'I wish java could use case classes'),
    (2,'Logistic,regression,models,are,neat')
],['id','sentence'])

In [6]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [7]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [9]:
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words',
                                pattern='\\W')

In [10]:
count_tokens = udf(lambda words:len(words),IntegerType())

In [12]:
tokenized = tokenizer.transform(sen_df)

In [13]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [14]:
tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [15]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [16]:
rg_tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [17]:
from pyspark.ml.feature import StopWordsRemover

In [19]:
sentenceDataFrame = spark.createDataFrame([
    (0,['I','saw','the','green','horse']),
    (1,['Mary','had','a','little','lamb'])
],['id','tokens'])

In [20]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [21]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')

In [22]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [30]:
from pyspark.ml.feature import NGram

In [35]:
wordDataFrame = spark.createDataFrame([
    (0,['Hi', 'I', 'heard', 'about', 'spark']),
    (1,['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2,['Logistic', 'regression', 'models', 'are', 'neat'])
],['id', 'words'])

In [36]:
wordDataFrame.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|[Hi, I, heard, ab...|
|  1|[I, wish, java, c...|
|  2|[Logistic, regres...|
+---+--------------------+



In [37]:
ngram = NGram(n=2,inputCol='words', outputCol='grams')

In [38]:
ngram.transform(wordDataFrame).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, java, c...|[I wish, wish jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



In [39]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [40]:
ngram = NGram(n=3,inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[Hi I heard, I heard about, heard about spark]                                  |
|[I wish java, wish java could, java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+



In [41]:
ngram = NGram(n=2,inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [42]:
from pyspark.ml.feature import HashingTF,IDF

In [64]:
sentenceData = spark.createDataFrame([
    (0.0,'Hi I heard about spark'),
    (0.0,'I wish java could use case classes'),
    (1.0,'Logistic,regression,models,are,neat')
],['label','sentence'])

In [65]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic,regressi...|
+-----+--------------------+



In [66]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [67]:
words_data = tokenizer.transform(sentenceData)

In [68]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |
+-----+-----------------------------------+------------------------------------------+



In [69]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [70]:
featurized_data = hashing_tf.transform(words_data)

In [71]:
featurized_data.show()

+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[24417,49...|
|  0.0|I wish java could...|[i, wish, java, c...|(262144,[20719,24...|
|  1.0|Logistic,regressi...|[logistic,regress...|(262144,[84234],[...|
+-----+--------------------+--------------------+--------------------+



In [72]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [73]:
idf_model = idf.fit(featurized_data)

In [74]:
rescaled_data = idf_model.transform(featurized_data)

In [78]:
rescaled_data.select('label','features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|0.0  |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [56]:
from pyspark.ml.feature import CountVectorizer

In [57]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ['id', 'words'])

In [58]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [79]:
cv = CountVectorizer(inputCol='words', outputCol='features',
                    vocabSize=3,minDF=2.0)

In [80]:
model = cv.fit(df)

In [81]:
result = model.transform(df)

In [82]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

